In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [14]:
import pandas as pd
import tensorflow as tf
from pycocotools.coco import COCO
import cv2
import os

In [15]:
## training
dmg_img_path = './Dataset/1.Training/1.원천데이터/TS_damage/damage/'
part_img_path = './Dataset/1.Training/1.원천데이터/damage_part/'

dmg_json_path = './Dataset/1.Training/2.라벨링데이터/damage/'
part_json_path = './Dataset/1.Training/2.라벨링데이터/damage_part/'

In [16]:
# vailidation
val_dmg_img_path = './Dataset/2.Validation/1.원천데이터/VS_damage/damage/'
val_part_img_path = './Dataset/2.Validation/1.원천데이터/damage_part/'

val_dmg_json_path = './Dataset/2.Validation/2.라벨링데이터/damage/'
val_part_json_path = './Dataset/2.Validation/2.라벨링데이터/damage_part/'

In [17]:
dmg_img = os.listdir(dmg_img_path)
dmg_json = os.listdir(dmg_json_path)

part_img = os.listdir(part_img_path)
part_json = os.listdir(part_json_path)

In [21]:
# val_dmg_img = os.listdir(val_dmg_img_path)
# val_dmg_json = os.listdir(val_dmg_json_path)

# val_part_img = os.listdir(val_part_img_path)
# val_part_json = os.listdir(val_part_json_path)

In [22]:
### set으로 한 결과도 같으므로 이름이 전부 다르다
len(dmg_img)
len(set(dmg_img))
len(dmg_json)
len(set(dmg_json))

402143

402143

403559

403559

In [23]:
len(val_dmg_img)
# len(set(val_dmg_img))
len(val_dmg_json)
# len(set(val_dmg_json))

len(val_part_img)
len(val_part_json)

19959

50445

17248

17248

In [24]:
print('dmg_img_name')
for i in range(3):
    dmg_img[i].split('.')[0]

print('dmg_json_name')
for i in range(3):
    dmg_json[i].split('.')[0]

dmg_img_name


'0000002_as-0036229'

'0000003_as-0036229'

'0000004_as-0036229'

dmg_json_name


'0000002_as-0036229'

'0000003_as-0036229'

'0000004_as-0036229'

In [25]:
dmg_json_copy = dmg_json.copy()
dmg_json_copy[:5]

['0000002_as-0036229.json',
 '0000003_as-0036229.json',
 '0000004_as-0036229.json',
 '0000005_as-0036229.json',
 '0000006_as-0036229.json']

### 문제점
1. img파일이 있는 폴더와 json파일이 있는 폴더가 있다
2. img파일과 json파일의 이름은 대부분 확장자를 제외하고 같지만 그렇지 않는 파일이 있다.
3. img파일에는 존재하는 이름이지만 json파일에는 존재하지 않는 파일은 삭제한다.
4. json파일에는 존재하지만 img파일에는 존재하지 않는 파일도 삭제한다.

- img_dmg_path = './Dataset/1.Training/1.원천데이터/TS_damage/damage/'
- img_part_path = './Dataset/1.Training/1.원천데이터/damage_part/'

- json_dmg_path = './Dataset/1.Training/2.라벨링데이터/damage/'
- json_part_path = './Dataset/2.Validation/2.라벨링데이터/damage_part/'

### 해결 방식
1. 이미지 파일명을 .jpg를 제외하여 배열로 가져온다
2. json 파일명을 .json를 제외하고 배열로 가져온다
3. 이미지 배열의 첫 번째값과 json배열 전체를 비교한다
    - 같은 값이 존재하면 해당 배열의 값을 지운다 (img,json 모두)
4. 각각의 배열에서 이름이 동시에 존재하는 배열이 전부 지워진다
5. 남아 있는 배열과 같은 이름의 이미지 파일과 json파일을 제거한다    

In [26]:
# 집합을 이용

list_img = ['a','b','c','d','f','g','i']
# list_img_copy = list_img.copy()
list_json = ['a','b','c','d','e','g','h']
# list_json_copy = list_json.copt()

complement = list(set(list_img) - set(list_json))
complement
complement2 = list(set(list_json) - set(list_img))
complement2

['f', 'i']

['h', 'e']

In [27]:
def difference_del(list1, list2):
    list1_split = [x.split('.')[0] for x in list1]
    list2_split =  [x.split('.')[0] for x in list2]
    list1_difference = list(set(list1_split) - set(list2_split))
    list2_difference = list(set(list2_split) - set(list1_split))

    return list1_difference, list2_difference

In [28]:
dmg_img_difference, dmg_json_difference = difference_del(dmg_img,dmg_json)

# dmg_img_difference[:10]
# dmg_json_difference[:10]
len(dmg_img_difference)
len(dmg_json_difference)

0

1416

In [29]:
part_img_difference, part_json_difference = difference_del(part_img,part_json)
len(part_img_difference)
len(part_json_difference)

len(part_img)
len(part_json)

0

0

109062

109062

In [31]:
len(dmg_img)
len(dmg_json)

402143

403559

In [35]:
# 이미지에는 없는 json파일 : 지워야 하는 파일
dmg_json_extra = [x+'.json' for x in dmg_json_difference]
cnt = 0
for i in dmg_json:
    if i in dmg_json_extra:
        print(dmg_json_path + i)
        os.remove(dmg_json_path + i)

./Dataset/1.Training/2.라벨링데이터/damage/0000524_as-0081945.json
./Dataset/1.Training/2.라벨링데이터/damage/0000542_sc-156063.json
./Dataset/1.Training/2.라벨링데이터/damage/0000664_sc-222505.json
./Dataset/1.Training/2.라벨링데이터/damage/0000957_sc-227933.json
./Dataset/1.Training/2.라벨링데이터/damage/0000958_sc-227933.json
./Dataset/1.Training/2.라벨링데이터/damage/0000961_sc-227933.json
./Dataset/1.Training/2.라벨링데이터/damage/0001024_sc-1030723.json
./Dataset/1.Training/2.라벨링데이터/damage/0001032_as-0049570.json
./Dataset/1.Training/2.라벨링데이터/damage/0001107_sc-213034.json
./Dataset/1.Training/2.라벨링데이터/damage/0001562_sc-201252.json
./Dataset/1.Training/2.라벨링데이터/damage/0001575_sc-157444.json
./Dataset/1.Training/2.라벨링데이터/damage/0001662_sc-160846.json
./Dataset/1.Training/2.라벨링데이터/damage/0001664_sc-160846.json
./Dataset/1.Training/2.라벨링데이터/damage/0001665_sc-160846.json
./Dataset/1.Training/2.라벨링데이터/damage/0001686_sc-137447.json
./Dataset/1.Training/2.라벨링데이터/damage/0001931_sc-216277.json
./Dataset/1.Training/2.라벨링데이터/damage/

In [36]:
#img 파일에 존재하지 않는 json파일 지우기
# dmg_json_remove = [x+'.json' for x in dmg_json_difference]

In [19]:
# import json
# dmg_json_file = [dmg_json_path+x for x in dmg_json]

# # for i in range(len(dmg_json)):
# for i in range(400000):
#     with open(dmg_json_file[i], 'r') as f:
#         cnt=0
#         json_data = json.load(f)
#         if json_data['images']['file_name'] == dmg_json[i]:
#             pass
#         else:
#             cnt+=1
# cnt
